# `dgaf` tests


one way to use dgaf is it with the `doit` line magic; load the `dgaf` ipython extension with

In [1]:
    %reload_ext src.dgaf

to reveal this mode.

below we generate a list of develop the tasks `dgaf` is configured with:

In [2]:
%doit list

blog               infer the nikola blog documentation configuration.
build              build the python project.
conda              install conda requirements
config             infer the jupyter_book documentation configuration.
develop            install the project in development mode.
environment_yaml   infer the project dependencies and write them to an environment.yaml
install            install the packages into the sys.packages
jupyter_book       
mkdocs             build the documentation with mkdocs
mkdocs_yml         infer the mkdocs documentation configuration.
nikola             build the documentation with nikola
pip                install pip requirements
pyproject          infer the pyproject.toml configuration for the project
requirements_txt   infer the project dependencies and write them to a requirements.txt
setup_cfg          infer the declarative setup.cfg configuration for the project
sphinx             build the documentation with sphinx
test               
to

we forget all of our tasks for testing purposes, this is a feature for the `doit` command line that we rely entirely on.

In [3]:
%doit forget

forgetting all tasks


below we ask information about the `pyproject` task.

In [4]:
%doit info pyproject


pyproject

infer the pyproject.toml configuration for the project

status     : run
 * The following file dependencies have changed:
    - requirements.txt

file_dep   : 
 - requirements.txt

targets    : 
 - pyproject.toml

params     : 
 - Param(name='backend', default='flit', long=None, short=None, type=None, help=None, choices=(('flit', 'flit'), ('poetry', 'poetry'), ('setuptools', 'setuptools')))


the `pyproject` task depends on the `requirements.txt` that contains the project dependencies; this information is joined with heuristics that infer project metadata to configure the setup of `flit, poetry and setuptools` projects.

when we run the project the first time, more than one task is run because of the file dependencies defined for each task.

In [5]:
%doit pyproject

MyReporter --> requirements_txt
MyReporter --> pyproject


the -s flag can run a single task

In [6]:
%doit -s pyproject

-- pyproject


running the command again we notice that the `--` indicating that the task is skipped because no dependencies are updated.

In [7]:
%doit pyproject

MyReporter --> requirements_txt
-- pyproject


In [8]:
%doit build

MyReporter --> requirements_txt
-- pyproject
MyReporter --> build
flit_core
TaskError - taskid:build
PythonAction Error
Traceback (most recent call last):
  File "/home/tonyfast/miniconda3/lib/python3.7/site-packages/doit/action.py", line 437, in execute
    returned_value = self.py_callable(*self.args, **kwargs)
  File "/home/tonyfast/Documents/dgaf/src/dgaf/install.py", line 73, in build
    assert not doit.tools.CmdAction("flit build").execute()
AssertionError



In [1]:
!flit build

>?47h8Version number normalised: '2021.01.11' -> '2021.1.11' (see PEP 440)  W-flit_core.versionno
Untracked or deleted files in the source directory. Commit, undo or ignore these files in your VCS.


In [9]:
%doit info build


build

build the python project.

status     : run
 * The following targets do not exist:
    - dist/dgaf-2021.1.11-py3-none-any.whl
    - dist/dgaf-2021.1.11.tar.gz
 * The following file dependencies have changed:
    - pyproject.toml

file_dep   : 
 - pyproject.toml

targets    : 
 - dist/dgaf-2021.1.11-py3-none-any.whl
 - dist/dgaf-2021.1.11.tar.gz

params     : 
 - Param(name='develop', default=True, long=None, short=None, type=<class 'bool'>, help='use development tools to build the project.', choices=())
 - Param(name='pip', default=False, long=None, short=None, type=<class 'bool'>, help='build with generic standard python packaging tools.', choices=())


    [tool.dgaf]
    id = "<url>"
    infer = False
    build = "flit"
    install = "conda"
    develop = True
    docs = "jupyter-book"
    lint = "flakehell/pre-commit"
    name = "tonyfast"
    email = "tony.fast@gmail.com"
    venv = True

## test functions

`inference` tests the interactive `dgaf` api for its ability to generate the proper configurations for the `project`.

In [1]:
    def inference(pytester, project):
        assert project.is_flit()
        assert project.get_name() == "my_idea"
        assert project.get_description() == "my projects docstring"
        assert project.get_version() == "0.0.1"
        assert project.get_requires() == ["pandas"]

`configuration` tests the ability for the `project` to configure itself. the primary intent of `dgaf` is distribution literate programs as literature and software. below we test the abilities to:

* generate pyproject.toml file for building the project.
* generate table of contents and documentation configuration.
* configure the linters and formatters

a lot of information can be encoded in a the `pyproject.toml` file. `dgaf` configures `flakehell, pytest and flit` in these tests.

In [9]:
    def configuration(pytester, project):
        project.add("python")
        assert (project/ "pyproject.toml").exists()
        
        project.add("docs")
        assert (project/ "docs/_toc.yml").exists() and (project/ "docs/_config.yml").exists()
        
        project.add("lint")
        assert (project / '.pre-commit-config.yaml').exists()        

`verify` checks the ability of different tools to use the generated configurations.

the conventions we choose have either implicit/explicit schema and uses consumers of those schema to verify the configuration has the appropriate forms. we do not explicitly verify schema with `dgaf`.

In [10]:
    def verify(pytester, project):
        import dgaf
        pyproject = dgaf.File(project/ "pyproject.toml").load()
        print(pyproject)
        tool = pyproject.get("tool", {})
        assert all(x in tool for x in "flit pytest".split())
        assert not pytester.run(*"flit build --format sdist".split()).ret

## the test objects

`build` creates a test directory from nested dictionaries.

In [11]:
    import pathlib, pytest
    def build(pytester, object, where=None):
        for key, value in object.items():
            if isinstance(value, str):
                file = (where or pathlib.Path()) / key
                if where:
                    where.mkdir(exist_ok=True, parents=True)
                if file.suffix == ".ipynb":
                    import nbformat
                    value = nbformat.v4.writes(nbformat.v4.new_notebook(cells=[nbformat.v4.new_code_cell(contents)]))
                pytester.makefile(file.suffix, **{
                    str(file.with_suffix("")): value
                })
            elif isinstance(value, dict):
                build(pytester, value, where=(where or pathlib.Path())/key)

for this test document we'll consider a simple project with the contents below. in the `contents`, we need to explicitly provide a docstring and version to cooperate with `flit`s model.

In [12]:
    contents = """'''my projects docstring'''
    __version__ = "0.0.1"
    
    import pandas
    """

it allows different layouts, like `python_layouts` to be used as test input.

In [13]:
    python_layouts = [{
        "my_idea.py": contents
    }, dict(
        my_idea={
            "__init__.py": contents
        }
    ), dict(
        src=dict(
            my_idea={
                "__init__.py": contents
            }
        )
    ), {
        "my_idea.ipynb": contents
    }]

In [14]:
    @pytest.mark.parametrize("layout", python_layouts)
    def test_python(pytester, layout):
        import dgaf
        build(pytester, layout)
        project = dgaf.Project(pytester.path)
        configuration(pytester, project)
        project = dgaf.Project(pytester.path)
        assert project.is_flit()
        inference(pytester, project)
        verify(pytester, project)

## test the `src.dgaf` directly.

In [1]:
    import dgaf

'dgaf'

In [15]:
    def test_dgaf_project():
        from src import dgaf        
        self = project = dgaf.Project()
        assert project.get_name() == "dgaf" == dgaf.Project("src").get_name()
        assert project.is_flit()
        assert project.get_description()
        requires = ["nox", "typer"]
        print(project.get_requires())
        
        assert project.get_requires() == ["nox", "typer"]
        assert project.get_url()
        assert "@" in project.get_email()
        assert 'docs/_build/' in project.get_exclude()